# Image classification with Convolutional Neural Networks
## Sign Language

Date: 1st May 2018

The idea is to make a Deep Learning model that can predict the sign language. To start with in phase 1 I'll just train it to identify the 10 digits 1-9

http://www.lifeprint.com/asl101/pages-signs/n/numbers.htm

To create samples of the images. I have used OpenCV and captured the images via the laptops camera. 
These images were then uploaded to the Paperspace VM

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.backends.cudnn.enabled

True

The next few steps will create the train & test dataset from my original dataset. 
Date: 01st May 2018 :: As of now only a few images for 1-5 have been uploaded to see if I can get a minimum viable product (MVP)
Later will add more images to complete the entire set ( a-zA-Z and 0-9)

In [6]:
#########################################################################################
##  This program is to perform predictions on the images

#########################################################################################

In [7]:
import os 

PATH = f'C:\\OpenCV\\Project Folder\\'

In [8]:
def get_data(sz,bs):
    tfms=tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom =1.3)
    data = ImageClassifierData.from_paths(PATH, tfms=tfms, trn_name='train', val_name = 'test', num_workers = 4)
    return data if sz > 300 else data.resize(340, 'tmp')
    #return data

Before using a different architecture don't forget to download the pre-computed weights into the weights folder
wget http://files.fast.ai/models/weights.tgz

In [9]:
# Uncomment the below if you need to reset your precomputed activations
#shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

In [11]:

# Cleaning directories 
import os
from shutil import copyfile
import random


# Create train & test folders

if not os.path.exists(f'{PATH}train'):
    os.makedirs(f'{PATH}train')
if not os.path.exists(f'{PATH}test'):
    os.makedirs(f'{PATH}test')
    


# Get all the classes in the directory
trainClasses = os.listdir(f'{PATH}signs\\')
trainClasses

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H']

In [15]:

for classes in trainClasses:
    fileName = os.listdir(f'{PATH}signs\\'+classes)
    print("Class: ", classes)
    print(len(fileName))
    sizeTrain = int(len(fileName)*0.8)
    
    random.shuffle(fileName)
    
    # Create folders for dataPoints
    if not os.path.exists(f'{PATH}train\\'+classes):
        os.makedirs(f'{PATH}train\\'+classes)
    if not os.path.exists(f'{PATH}test\\'+classes):
        os.makedirs(f'{PATH}test\\'+classes)
    
    for idx in range(len(fileName)):
        
        srcPath = PATH+ 'signs\\' + classes+'\\' + fileName[idx]
        if idx < sizeTrain:
            dstPath = PATH +'train\\' + classes +'\\'+fileName[idx]
        else:
            dstPath = PATH +'test\\' + classes +'\\'+fileName[idx]
            
        copyfile(srcPath, dstPath)

Class:  0
477
Class:  1
521
Class:  2
795
Class:  3
859
Class:  4
884
Class:  5
582
Class:  6
343
Class:  7
604
Class:  8
529
Class:  9
692
Class:  A
817
Class:  B
0
Class:  C
0
Class:  D
0
Class:  E
0
Class:  F
0
Class:  G
0
Class:  H
0


In [26]:
#PATH = f'signs/'
bs =  8
sz = 120 # From above data vizualization on image size
arch= resnext101_64  ## resnet34 - This was giving an accuracy of around 60%

In [27]:
data = get_data(sz, bs)  

In [35]:
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [36]:
learn.load('C:\\OpenCV\\Project Folder\\models\\cax_final_01052018_1852')

RuntimeError: While copying the parameter named 0.weight, whose dimensions in the model are torch.Size([64, 3, 7, 7]) and whose dimensions in the checkpoint are torch.Size([4096]).

In [19]:
fileName = os.listdir(PATH+'test//5//')
img = open_image(PATH+'test//5//'+ fileName[5])
#plt.imshow(img)

In [20]:
trn_tfms, val_tfms = tfms_from_model(arch,sz) # get transformations
im = val_tfms(img)
learn.precompute=False # We'll pass in a raw image, not activations
preds = learn.predict_array(im[None])
np.argmax(preds) # preds are log probabilities of classes

9

In [ ]:
lrf=learn.lr_find()
learn.sched.plot_lr()

In [ ]:
learn.sched.plot() # For classes 1-5: Using 0.1 for the learning rate
                    # For classes 0-9: Again using 0.08 for the learning rate

In [ ]:
lrf = 0.08 
%time learn.fit(lrf, 4 )

In [ ]:
lrf = 0.08
%time learn.fit(lrf, 3 ) # The validation and training loss are very encouraging!
                         # 2      0.035845   0.037668   0.997024 

In [ ]:
%time learn.fit(lrf, 4, cycle_len=1, cycle_mult=3 )

The model seems good. as training and test errors are quite similar 

39     0.018122   0.023692   0.997768

In [ ]:
# 1st May 2018 - Saving model
learn.save('cax_final_01052018_2130')

Creating a confusion matrix

In [ ]:
learn.load('cax_final_01052018_2130')
log_preds,y = learn.TTA()

probs = np.mean(np.exp(log_preds),0)
preds = np.argmax(probs, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)
plot_confusion_matrix(cm, data.classes)

In [ ]:
fileName = os.listdir(PATH+'test//5//')
img = open_image(PATH+'test//5//'+ fileName[5])
#plt.imshow(img)

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch,sz) # get transformations
im = val_tfms(img)
learn.precompute=False # We'll pass in a raw image, not activations
preds = learn.predict_array(im[None])
np.argmax(preds) # preds are log probabilities of classes